### Function definitions (Algorithms)

In [1]:
from itertools import combinations, chain
import random

def compute_closure(attributes, fds) -> set:
    """
    Compute the closure of a set of attributes under a set of functional dependencies
    ---------------------------------------------------------------------------------
    attributes: a set of attributes
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    closure = set(attributes)
    changed = True
    while changed:
        changed = False
        for fd in fds:
            if fd[0].issubset(closure) and not fd[1].issubset(closure):
                closure.update(fd[1])
                changed = True
    return closure

def compute_all_closures(attributes, fds) -> dict:
    """
    Compute the closure of all possible subsets of a set of attributes
    ------------------------------------------------------------------
    attributes: a set of attributes
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    all_closures = {}
    for r in range(1, len(attributes) + 1):
        for subset in combinations(attributes, r):
            subset_closure = compute_closure(set(subset), fds)
            all_closures[tuple(subset)] = subset_closure
    return all_closures

def compute_candidate_keys(closure_set, attributes) -> list:
    """
    Compute the candidate keys of a set of attributes
    -------------------------------------------------
    closure_set: a dictionary of all closures
    attributes: a set of attributes
    """
    super_keys = []
    for i in closure_set:
        if set(closure_set[i]) == set(attributes):
            super_keys.append(i)
    candidate_keys = []
    for j in super_keys:
        flag = False
        for i in super_keys:
            if set(i) != set(j):
                if set(i).issubset(set(j)):
                    flag = True
        if flag == False:
            candidate_keys.append(j)
    return candidate_keys

def find_prime_attributes(candidate_keys) -> set:
    """
    Find the prime attributes of a set of candidate keys
    ----------------------------------------------------
    candidate_keys: a list of candidate keys
    """
    prime_attributes = set()
    for key in candidate_keys:
        prime_attributes.update(key)
    return prime_attributes

def compute_single_covers(attributes, fds) -> dict:
    """
    Compute the closure of each attribute in a set of attributes
    ------------------------------------------------------------
    attributes: a set of attributes
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    all_closures = {}
    for a in attributes:
        subset_closure = compute_closure(a, fds)
        all_closures[a] = subset_closure
    return all_closures

def project_dependency(fds, R_hat) -> list:
    """
    Project a set of functional dependencies on a set of attributes
    ---------------------------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    R_hat: a set of attributes
    """
    fds_hat = []
    for fd in fds:
        if fd[0].issubset(R_hat):
            y = fd[1].intersection(R_hat)
            if len(y)>0:
                fds_hat.append((fd[0],y))
    return fds_hat

## Minimal cover computation

def decompose_fds(fds) -> list:
    """Decompose each FD so that the RHS contains only one attribute.
    For example, the FD {A} -> {B, C} will be decomposed into {A} -> {B} and {A} -> {C}.
    ------------------------------------------------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    decomposed_fds = []
    for lhs, rhs in fds:
        for attr in rhs:
            decomposed_fds.append((lhs, {attr}))
    return decomposed_fds

def remove_trivial_dependencies(fds) -> list:
    """Remove trivial FDs of the form A -> A.
    -----------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    return [(lhs, rhs) for lhs, rhs in fds if lhs != rhs]

def remove_redundant_dependencies(fds) -> list:
    """Remove redundant FDs by checking if we can infer a FD from others.
    ---------------------------------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    non_redundant_fds = []
    for i, (lhs, rhs) in enumerate(fds):
        remaining_fds = fds[:i] + fds[i+1:]
        closure_lhs = compute_closure(lhs, remaining_fds)
        
        if not rhs.issubset(closure_lhs):
            non_redundant_fds.append((lhs, rhs))
    
    return non_redundant_fds

def remove_redundant_dependencies(fds) -> list:
    """Remove redundant FDs by checking if we can infer a FD from others.
    ---------------------------------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    len_fds_1 = len(fds)
    len_fds_2 = 0
    while len_fds_1>len_fds_2:
        len_fds_1 = len(fds)
        for i, (lhs, rhs) in enumerate(fds):
            remaining_fds = fds[:i] + fds[i+1:]
            closure_lhs = compute_closure(lhs, remaining_fds)
            if rhs.issubset(closure_lhs):
                fds.remove((lhs, rhs))
        len_fds_2 = len(fds)
    return fds

def merge_fds(fds) -> list:
    """Merge FDs with the same LHS back together.
    --------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    merged_fds = {}
    for lhs, rhs in fds:
        lhs = tuple(lhs)
        if lhs in merged_fds:
            merged_fds[lhs].update(rhs)
        else:
            merged_fds[lhs] = set(rhs)
    
    return [(set(lhs), rhs) for lhs, rhs in merged_fds.items()]

def powerset(iterable):
    """Generate all non-empty proper subsets of a set."""
    s = list(iterable)
    combs = [[i for i in combinations(s, r)] for r in range(1, len(s)+1)]
    return [x for xs in combs for x in xs]

def remove_superfluous_lhs(fds, p):
    """
    Simplify the LHS by checking if any proper subset of the LHS can imply the RHS.
    --------------------------------------------------------------------------------
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    minimal_fds = []
    for lhs, rhs in fds:
        minimal_lhs = lhs
        min_sub = 10000
        minimals = []
        for subset in powerset(lhs):
            if len(subset) <= min_sub:
                if rhs.issubset(compute_closure(set(subset), fds)):
                    minimal_lhs = set(subset)
                    min_sub = len(subset)
                    minimals.append(minimal_lhs)
        if len(minimals)>1 and random.randint(0, 10) < p*10:
            minimal_lhs = set(random.choice(minimals))
        else:
            minimal_lhs = minimals[0]
            
        minimal_fds.append((minimal_lhs, rhs))
    return minimal_fds

def minimal_cover(fds, p = 0.5) -> list:
    """Find the minimal cover of a set of FDs.
    -----------------------------------------
    attributes: a set of attributes
    fds: a list of functional dependencies (contains tuples of two sets. First set implies the second set)
    """
    # Step 1: Decompose the RHS
    decomposed_fds = decompose_fds(fds)

    # Step 2: Simplify LHS
    simplified_fds = remove_superfluous_lhs(decomposed_fds, p)

    # Step 3: Remove trivial dependencies (A -> A)
    simplified_fds = remove_trivial_dependencies(simplified_fds)

    # Step 4: Remove redundant FDs
    simplified_fds = remove_redundant_dependencies(simplified_fds)
    
    # Step 5: Recollect FDs with the same LHS
    minimal_fds = merge_fds(simplified_fds)
    
    return minimal_fds

### Example usage

In [2]:
attributes = {'A', 'B', 'C', 'D', 'E'}
fds = [
    ({'A'}, {'A', 'B', 'C'}),
    ({'A', 'B'}, {'A'}),
    ({'B', 'C'}, {'A', 'D'}),
    ({'B'}, {'A', 'B'}),
    ({'C'}, {'D'})
]

In [3]:
all_closures = compute_all_closures(attributes, fds)

In [4]:
for k in all_closures:
    print('{k}+ = {v}'.format(k=set(k), v=all_closures[k]))

{'E'}+ = {'E'}
{'C'}+ = {'C', 'D'}
{'A'}+ = {'C', 'A', 'B', 'D'}
{'B'}+ = {'C', 'A', 'B', 'D'}
{'D'}+ = {'D'}
{'E', 'C'}+ = {'E', 'C', 'D'}
{'E', 'A'}+ = {'E', 'C', 'A', 'B', 'D'}
{'E', 'B'}+ = {'E', 'C', 'A', 'B', 'D'}
{'E', 'D'}+ = {'E', 'D'}
{'A', 'C'}+ = {'C', 'A', 'B', 'D'}
{'B', 'C'}+ = {'A', 'B', 'C', 'D'}
{'C', 'D'}+ = {'C', 'D'}
{'A', 'B'}+ = {'C', 'A', 'B', 'D'}
{'A', 'D'}+ = {'C', 'A', 'B', 'D'}
{'B', 'D'}+ = {'C', 'A', 'B', 'D'}
{'E', 'A', 'C'}+ = {'E', 'C', 'A', 'B', 'D'}
{'E', 'B', 'C'}+ = {'E', 'C', 'A', 'B', 'D'}
{'E', 'C', 'D'}+ = {'E', 'C', 'D'}
{'E', 'A', 'B'}+ = {'E', 'C', 'A', 'B', 'D'}
{'E', 'A', 'D'}+ = {'E', 'C', 'A', 'B', 'D'}
{'E', 'B', 'D'}+ = {'E', 'C', 'A', 'B', 'D'}
{'A', 'B', 'C'}+ = {'C', 'A', 'B', 'D'}
{'A', 'C', 'D'}+ = {'C', 'A', 'B', 'D'}
{'B', 'C', 'D'}+ = {'C', 'A', 'B', 'D'}
{'A', 'B', 'D'}+ = {'C', 'A', 'B', 'D'}
{'E', 'A', 'B', 'C'}+ = {'E', 'C', 'A', 'B', 'D'}
{'E', 'A', 'C', 'D'}+ = {'E', 'C', 'A', 'B', 'D'}
{'E', 'B', 'C', 'D'}+ = {'E', 'C', 

In [5]:
compute_candidate_keys(all_closures, attributes)

[('E', 'A'), ('E', 'B')]

In [6]:
find_prime_attributes(compute_candidate_keys(all_closures, attributes))

{'A', 'B', 'E'}

In [7]:
minimal_fds = minimal_cover(fds, p = 0.4)
for lhs, rhs in minimal_fds:
    print(f"{lhs} -> {rhs}")

{'A'} -> {'B', 'C'}
{'B'} -> {'A'}
{'C'} -> {'D'}


### Dependency projection example

In [8]:
fds = [
    ({'A'}, {'A', 'B', 'C'}),
    ({'A', 'B'}, {'A'}),
    ({'B', 'C'}, {'A', 'D'}),
    ({'B'}, {'A', 'B'}),
    ({'C'}, {'D'})
]

R_hat = {'A', 'B'}

In [9]:
project_dependency(fds, R_hat)

[({'A'}, {'A', 'B'}), ({'A', 'B'}, {'A'}), ({'B'}, {'A', 'B'})]